In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import math
import requests
import json
import re
from selenium.webdriver import Firefox, Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.page_load_strategy = 'eager'
#options.headless = True
options.add_argument("--start-maximized")
options.add_argument("--disable-infobars")
options.add_argument("--window-size=1920x1080")

In [11]:
driver = Chrome(executable_path="C:\\Users\\USER\\Downloads\\chromedriver.exe", options=options)
driver.maximize_window()

In [3]:
driver.get("https://www.zomato.com/sydney/restaurants")
#wait = WebDriverWait(driver, 20)
#element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h6.sc-elhb8j-8.fvpjYk")))
time.sleep(10)

In [4]:
footer = driver.find_element_by_css_selector("footer")
for _ in range(8):
    driver.execute_script("arguments[0].scrollIntoView();", footer)
    time.sleep(1)
    
time.sleep(60)

In [5]:
source = driver.page_source

In [6]:
#driver.close()
#driver.quit()

In [7]:
soup = BeautifulSoup(source, 'html.parser')

In [8]:
link_tags = soup.select("div.jumbo-tracker a.sc-cgHJcJ.bUffyV")

In [9]:
links = [tag["href"] for tag in link_tags]

In [10]:
links = ["https://www.zomato.com" + link for link in links]

In [11]:
len(links)

492

In [12]:
def get_soup(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"})

    soup = BeautifulSoup(r.text, 'html.parser')
    
    return soup

In [13]:
def get_ssoup(url):
    driver.get(url)
    wait = WebDriverWait(driver, 20)
    element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "main > div > section:nth-child(2)")))
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    return soup

In [14]:
res_list = []

In [15]:
count = 0
for url in links:
    count +=1
    print(count)
    res_soup = get_ssoup(url)
    
    try:
        name = res_soup.select("h1.sc-7kepeu-0.sc-kafWEX.kTxZkY")[0].text
    except:
        name = ""

    try:
        cuisines = res_soup.select("div.sc-eXEjpC.heiMdG")[0].text
        cuisines = cuisines.split(",")
    except:
        cuisines = ""

    try:
        url = url
    except:
        url = ""

    try:
        ratings = res_soup.select("div.sc-1q7bklc-1.cILgox")[0].text
    except:
        ratings = ""

    try:
        no_reviews = res_soup.select("div.sc-1q7bklc-8.kEgyiI")[0].text
        no_reviews = no_reviews.replace(",", "")
    except:
        no_reviews = ""

    try:
        phone_number = res_soup.select("p.sc-1hez2tp-0.fanwIZ")[0].text
    except:
        phone_number = ""

    try:
        address = res_soup.select("p.sc-1hez2tp-0.clKRrC")[0].text
    except:
        address = ""

    try:
        opening_time = res_soup.select("span.sc-bYSBpT.ekNthJ")[0].text
        opening_time = opening_time.split(" ")[-1]
    except:
        opening_time = ""

    try:
        description = res_soup.find(text="Known For").findNext("p").text
    except:
        description = ""

    try:
        known_for = res_soup.find('h3', text = re.compile('People Say This Place Is Known For')).findNext("p").text
        known_for = known_for.split(",")
        known_for = [_.strip() for _ in known_for]
    except:
        known_for = ""

    try:
        popular_dishes = res_soup.find('h3', text = re.compile('Popular Dishes')).findNext("p").text
        popular_dishes = popular_dishes.split(",")
        popular_dishes = [_.strip() for _ in popular_dishes]
    except:
        popular_dishes = ""

    try:
        average_cost = res_soup.find(text="Average Cost").findNext("p").text
    except:
        average_cost = ""

    try:
        more_info = res_soup.find(text="More Info").findNext("div").select("div.sc-bke1zw-1")
        more_info = [_.text for _ in more_info]
    except:
        more_info = ""

    try:
        chef_details = res_soup.find(text="CHEF DETAILS").findNext("div").text
    except:
        chef_details = ""
        
        
    res_dict = {
        "name": name,
        "cuisines": cuisines,
        "ratings": ratings,
        "no_reviews": no_reviews,
        "phone_number": phone_number,
        "address": address,
        "opening_time": opening_time,
        "description": description,
        "known_for": known_for,
        "popular_dishes": popular_dishes,
        "average_cost": average_cost,
        "more_info": more_info,
        "chef_details": chef_details,
        "url": url
    }    
    
    res_list.append(res_dict)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
df = pd.DataFrame(res_list)

In [17]:
df

,name,cuisines,ratings,no_reviews,phone_number,address,opening_time,description,known_for,popular_dishes,average_cost,more_info,chef_details,url
0,Chat Thai - Haymarket,"[Thai, Salad]",4.6,2013,+61292111808,"20 Campbell Street, Haymarket, Sydney",11am,Family-owned and organic popular Thai restaura...,"[Gastronomical Experience, Relaxed Atmosphere,...","[Massaman Curry, Pad Thai, Noodle, Desserts, P...",$55 for two people (approx.) Without alcohol,"[Breakfast, Takeaway Available, Wheelchair Acc...",,https://www.zomato.com/sydney/chat-thai-haymar...
1,Home Thai,"[Thai, Salad]",4.6,1508,+61292615058,"Shop 1-2, 299 Sussex Street, CBD, Sydney",11am,Modern and sleek design; open kitchen with col...,"[Ample Seating Area, Efficient Service, Prompt...","[Masaman Beef Curry, Paw Paw Salad, Pad Thai, ...",$60 for two people (approx.),"[Takeaway Available, BYO, Indoor Seating, Vege...",,https://www.zomato.com/sydney/home-thai-cbd/info
2,La Renaissance Cafe Patisserie,"[Desserts, Bakery, French, Patisserie, Caf...",4.8,895,+61292414878,"47 Argyle Street, The Rocks, Sydney",(Today),"Cakes, pastries, macarons and savouries to enj...","[Charging Ports, Student Crowd, Cafe, Location...","[Mille Feuille, Almond Croissant, Macaroons, P...",$40 for two people (approx.),"[Breakfast, Takeaway Available, Wheelchair Acc...",,https://www.zomato.com/sydney/la-renaissance-c...
3,Le Monde,[Modern Australian],4.5,642,+61292113568,"83 Foveaux Street, Surry Hills, Sydney",(Today),Vibrant cafe serving the Surry Hills community...,"[Cozy Place, Bathroom, Weekend Brunch, Great M...","[Potato Rosti, Sweet Potato Salad, Breakfast P...",$40 for two people (approx.),"[Breakfast, Takeaway Available, Gluten Free Op...",,https://www.zomato.com/sydney/le-monde-surry-h...
4,Black Star Pastry,"[Bakery, Desserts, Coffee and Tea]",4.6,1862,+61295578656,"277 Australia Street, Newtown, Sydney",(Today),"Established in 2008, Black Star bakery-café ha...","[Pretty Decor, Bread, Price, Atmosphere, Quali...","[Chocolate Raspberry, Watermelon Cake, Orange ...",$30 for two people (approx.),"[Breakfast, Takeaway Available, Wheelchair Acc...",,https://www.zomato.com/sydney/black-star-pastr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,Momiji,[Japanese],3.9,65,+61297442977,"96a Burwood Road, Burwood, Sydney",5pm,,"[Flavour, Price, Food, Service]",,$40 for two people (approx.),"[Breakfast, Takeaway Available, Indoor Seating...",,https://www.zomato.com/sydney/momiji-burwood/info
488,Verde,[Italian],4.2,86,+61293808877,"115 Riley Street, Darlinghurst, Sydney",6pm,,"[Decor, Great Food, Staff, Service]",,$110 for two people (approx.) Without alcohol,"[Takeaway Available, Full Bar Available, Kid F...",,https://www.zomato.com/sydney/verde-darlinghur...
489,Double Brew,"[Sandwich, Healthy Food, Cafe Food, Coffee ...",4.0,47,+61413683949,"33 York Street, CBD, Sydney",6am,,,,$25 for two people (approx.),"[Breakfast, Takeaway Available, Wheelchair Acc...",,https://www.zomato.com/sydney/double-brew-1-cb...
490,Stix & Pita,[Greek],4.0,44,+61295161222,"Shop 3, 216, Enmore, Sydney",4pm,,"[Great Food, Quality, Staff, Service]",,$30 for two people (approx.),"[Takeaway Available, Indoor Seating, No Alcoho...",,https://www.zomato.com/sydney/stix-pita-enmore...


In [18]:
df.to_excel("restaurants.xlsx", index=False)

In [19]:
#driver = Chrome(executable_path="C:\\Users\\USER\\Downloads\\chromedriver.exe", options=options)
#driver.maximize_window()

In [12]:
df = pd.read_excel("restaurants.xlsx")

In [13]:
urls = df["url"]
review_url = [link.replace("/info", "/reviews") for link in urls]

In [14]:
res_list2 = []

In [16]:
len(res_list2)

274

In [17]:
count = 274

for url, rev_url in zip(urls[274:], review_url[274:]):
    
    count += 1
    print(count)
    driver.get(url)
    wait = WebDriverWait(driver, 20)
    element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "main > div > section:nth-child(2)")))
    
    try:
        gal = driver.find_element_by_css_selector("main > div > section:nth-child(2) img")
        driver.execute_script("arguments[0].scrollIntoView();", gal)
        time.sleep(2)
    except:
        pass

    try:
        gallery_img = driver.find_elements_by_css_selector("main > div > section:nth-child(2) img")
        gallery_images = [img.get_attribute("src") for img in gallery_img] 
    except:
        gallery_images = ""
        
    print(gallery_images)
        
    try: 
        men = driver.find_element_by_xpath("//h3[text()[contains(.,'Menu')]]/../following-sibling::section/.//img")
        driver.execute_script("arguments[0].scrollIntoView();", men)
        time.sleep(2)
    except:
        pass
    
    try:
        menu_img = driver.find_elements_by_xpath("//h3[text()[contains(.,'Menu')]]/../following-sibling::section/.//img")
        menu_images = [img.get_attribute("src") for img in menu_img]
        menu_images = [img.split("?fit")[0] for img in menu_images]
    except:
        menu_images = []
        
    print(menu_images)
    
    driver.get(rev_url)
    wait = WebDriverWait(driver, 20)
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "main > div > section:nth-child(2)")))
    
    try:
        rev = driver.find_element_by_xpath("//h2[text()[contains(.,'Reviews')]]")
        driver.execute_script("arguments[0].scrollIntoView();", rev)
    except:
        pass
    
    full_comments = []

    try:
        comments = driver.find_elements_by_xpath("//h2[text()[contains(.,'Reviews')]]/../div[2]/./p")
        comments = [com.text.replace("\n", "") for com in comments if len(com.text) != 0]
        for com in comments:
            full_comments.append(com)
    except:
        pass
            
            
    res_dict2 = {
        "gallery_images": gallery_images,
        "menu_images": menu_images,
        "comments": full_comments
    }
    
    res_list2.append(res_dict2)
    

275
['https://b.zmtcdn.com/data/pictures/chains/4/16564944/ce6cfbf4e5a2471be51946e0b13c95b7_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/chains/4/16564944/b6302a93b4406627571313d9fa4df3ff.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/4/16564944/4789f85c22752b9b82320cf2441ba44c.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/4/16564944/d3b46d0e213deeb07f11e1bc35bdb8a2.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/944/16564944/96c5a48796c32abbf0c23e9242cc554c.jpg', 'https://b.zmtcdn.com/data/menus/944/16564944/7cf168cd776d69fc1b2d826a0f847053.jpg']
276
['https://b.zmtcdn.com/data/pictures/5/18220345/81980a440075f1e1fb361ca9ae5d14b5_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/5/1822034

['https://b.zmtcdn.com/data/menus/764/16566764/c39060f678330d631773bf1d02bf598d.jpg', 'https://b.zmtcdn.com/data/menus/764/16566764/b623fa01067c3dbeb1b94d6fe6af8438.jpg', 'https://b.zmtcdn.com/data/menus/764/16566764/1991dfb544ff22c44cde298c8d936eaa.jpg']
287
['https://b.zmtcdn.com/data/pictures/chains/4/15544634/0a58f0749ede1a4e786fbfae3733dc3c_featured_v2.png?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/4/15544634/9ab8d7ef719f6d351cdbf52b4dc1a653.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/4/15544634/766ec58ffb7003c5e0e3dc11ffb38dbc.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/4/15544634/03fcdc6d77067cac9246b24b2beeb6b8.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/634/15544634/c9a7d10109d3d39745a3fc20aa924e05.jpg']
288
['https://b.zmtcdn.com/data/pictures/chains/2/1

298
['https://b.zmtcdn.com/data/pictures/chains/2/16557112/924df8a8172eee402b38a9ca27c9ee4e_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/chains/2/16557112/d1c9934b4b324a08f991d1cdec76f272.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/2/16557112/ecdebc1084eace3e2cd95e9db467cfa5.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/2/16557112/924df8a8172eee402b38a9ca27c9ee4e.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/112/16557112/5fd9a99a430319404531303132e042ae.jpg']
299
['https://b.zmtcdn.com/data/pictures/5/16563495/ad086d356e71f3d987d8dd378a5ee802_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/reviews_photos/16d/c60f5d350b8864fe64d734107a0dd16d.jpg?output-format=webp&fit=around|300:273&cr

309
['https://b.zmtcdn.com/data/pictures/9/17742309/15b2c98bc588f9d54befd3cf4595f33e_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/chains/9/17742309/6a3af869a45b788bad58b8831a9731d3.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/9/17742309/cafe4342c0d1a40d274e58ad6c58f64c.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/9/17742309/98f8d560dfb4c07d7384c64beca8f19b.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/309/17742309/93ad795d61cb8900157f403a04c2019f.jpg']
310
['https://b.zmtcdn.com/data/pictures/chains/0/16559140/8a4531e587808df3859ea57800b9d4bb_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/0/16559140/be7875e9bda7cd382323b774805a6674.jpg?output-format=webp&fit=around|300:273&crop=300

321
['https://b.zmtcdn.com/data/reviews_photos/4e7/70292e5e571cadb5ed203510446cd4e7_1492146206.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/0/16558690/f2753bdf241804e2b483008f78e3a1d8.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/0/16558690/70dd283e5d314b56791ff2f28cf6869a.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/0/16558690/268518ef37e739bedbf14c4b5926517c.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/690/16558690/fb4858a06939c95df2a7025dde7a4fbb.jpg']
322
['https://b.zmtcdn.com/data/pictures/chains/4/16567364/3063cf7344e574fad17066ad7c332176_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/4/16567364/0d5669d6ffac8f984d26fdf10f038ea7.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'http

...

['https://b.zmtcdn.com/data/menus/539/15545539/a35a0fe293aa05dc8dbc9144f95c129c.jpg']
440
['https://b.zmtcdn.com/data/pictures/3/16561303/3e0708a74709d8a1744a19a669bc69eb_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/3/16561303/b1e3c4f66a77c49e050767b4b97bcf92.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/reviews_photos/a67/0665faa016f20dcbfb7043b6c3763a67_1542534309.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/reviews_photos/1d5/110c1060c4b3cfe206e3480bcedf21d5_1447188148.JPG?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/303/16561303/a90d7097c446ca155ce25397562656c0.jpg']
441
['https://b.zmtcdn.com/data/pictures/2/18318162/278c4f2ea0c7bc9ef473d38736918cdf_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/2/1831816

['https://b.zmtcdn.com/data/menus/692/16566692/2145c19aef248f9274a7efe5a73e210a.jpg', 'https://b.zmtcdn.com/data/menus/692/16566692/1860f3414283405301341c7b26f55453.jpg']
452
['https://b.zmtcdn.com/data/pictures/1/16557751/130ae5a02722ed57b47490224a1a0263_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/chains/1/16557751/cc9d2b95e985393b8023a283ea0ca6e2.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/1/16557751/8cd9d0a9eaa123d81647c91aa1a53486.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/1/16557751/6d04655deed9aeeb40c5cee62c769c24.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/751/16557751/8d1ecd15bf96c688b1c1752736ca8127.jpg']
453
['https://b.zmtcdn.com/data/reviews_photos/ec1/1888fc3c414601440e8e2e7c0c6b5ec1_1546410374.jpg?output-format=webp&fit=around|

['https://b.zmtcdn.com/data/menus/656/16566656/0fc31b27b208fe430dcad6bc54bcefa1.jpg', 'https://b.zmtcdn.com/data/menus/656/16566656/ccf0df7a922a5a24e8a39a7f12714e5e.jpg', 'https://b.zmtcdn.com/data/menus/656/16566656/4fa6360893898af093caecae85591ab4.jpeg', '']
464
['https://b.zmtcdn.com/data/pictures/chains/7/16564857/e8ffdbfe08eabd66e80673f5ff2bf1f6_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/7/16564857/59e759bd9829ee7e88ad6218737645cf.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/7/16564857/ef373f74154c1d35a4ae5d735571aabd.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/7/16564857/cddb44dda49c6a86f3e47c6202e8a6ad.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/857/16564857/64e3fb41ae6884212e43c5f3018bee49.jpg', 'https://b.zmtcdn.com/data/menus/857/16564857/e151e9a

475
['https://b.zmtcdn.com/data/pictures/chains/1/15545991/f0cc85082d7883cfadca3e2d6a6c4302_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/1/15545991/259c1e11ad0e49ca50c4ab9d4e3a41ea.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/1/15545991/d9ec1bcc4284a7ac038c4996e2d48106.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/1/15545991/0a5e2ca4c28d7407eb145e2c657d841a.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/991/15545991/d7da6346ac830d830982427a45614cc5.jpg']
476
['https://b.zmtcdn.com/data/pictures/6/19220466/9a1d20fa8825579d880e73a065d491d9.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/chains/6/19220466/a0a9ca50e26f51e344f3658bba6a177b.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https:/

['https://b.zmtcdn.com/data/menus/279/16559279/81e3f06f9cff8e230b07c755b4fe0334.jpg']
487
['https://b.zmtcdn.com/data/pictures/5/16564205/ab9000078ae33c0d06285d38f3ad1a96_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/5/16564205/833bca1487f4cbb1a0bb19ee0cf49b69.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/5/16564205/c8eb57af846bc58899c85058d35ff558.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*', 'https://b.zmtcdn.com/data/pictures/chains/5/16564205/a827f4fcf2093d811c9d802cdefbe0b2.jpg?output-format=webp&fit=around|300:273&crop=300:273;*,*']
['https://b.zmtcdn.com/data/menus/205/16564205/a36fe9f0ead01606eeec0356e29d15f3.jpg']
488
['https://b.zmtcdn.com/data/pictures/8/16567318/7e2c1d3f9bd911949ddd3b0e3a72e455_featured_v2.jpg?output-format=webp&fit=around|771.75:416.25&crop=771.75:416.25;*,*', 'https://b.zmtcdn.com/data/pictures/8/16567318/35cb

In [18]:
df2 = pd.DataFrame(res_list2)

In [20]:
df = pd.concat([df, df2], axis=1)

In [22]:
df.to_excel("restaurants2.xlsx", index=False)

In [23]:
driver.close()
driver.quit()